In [1]:
import pandas as pd
import requests 
import os

In [2]:
path="../SmallFiles/New/"

In [3]:
columns_names = ["Sample","Method","Coverage","Age", "NB_total_variant",
 "VARIANT_SNV","VARIANT_INDEL","VARIANT_Substitution","VARIANT_Insertion","VARIANT_Deletion","VARIANT_Others",
 "IMPACT_Modifier","IMPACT_Low","IMPACT_Moderate","IMPACT_HIGH",
 "SIFT_deletarious","SIFT_not_deletarious",
 "POLYPHEN_unknown","POLYPHEN_benign","POLYPHEN_possibly_damaging","POLYPHEN_probably_damaging",
 "GERP_constrained","GERP_not_constrained",
 "CADD_deletarious", "CADD_not_deletarious"]

In [7]:
def func(l): # function to count lines starting with ‘A’
    c = 0 # c is the count of such lines
    for line in l:
        if line[0] == "#":
            if line[1] == "#":
                c += 1
    return c

In [8]:
i=0
for filename in os.listdir(path):
    print(f"=== LOADING {filename} ===")
    file = os.path.join(path, filename)
    f1 = open(file, "r")
    l = f1.readlines() # creates a list of all the lines in the file
    lineskip = func(l)
    df = pd.read_csv(file, skiprows=int(lineskip)-1, delimiter="\t", header=0)
    f1.close()
    print(f"=== Treating {filename} ===")
    
    #find name
    Name = file.split("INFO_")[-1].split("_")[0]
    #find method
    method = file.split("INFO_")[-1].split("_")[2]
    
    #Adding values to list
    x=[]
    
    #SampleName
    x.append(Name)
    #Method
    x.append(method)
    #Coverage
    x.append("IDK")
    #Age
    x.append("IDKy")
    #Nb total of variant
    x.append(int(len(df)))
    
    #Variant Type
    #"VARIANT_SNV" - "VARIANT_INDEL" - "VARIANT_Substitution" - "VARIANT_Insertion" - "VARIANT_Deletion" - "VARIANT_Others"
    VARIANT_SNV = len(df[df["VARIANT_CLASS"]=="SNV"])
    x.append(VARIANT_SNV)
    VARIANT_INDEL = len(df[df["VARIANT_CLASS"]=="indel"])
    x.append(VARIANT_INDEL)
    VARIANT_Substitution = len(df[df["VARIANT_CLASS"]=="substitution"])
    x.append(VARIANT_Substitution)
    VARIANT_Insertion = len(df[df["VARIANT_CLASS"]=="insertion"])
    x.append(VARIANT_Insertion)
    VARIANT_Deletion = len(df[df["VARIANT_CLASS"]=="deletion"])
    x.append(VARIANT_Deletion)
    sumvar = VARIANT_SNV + VARIANT_INDEL + VARIANT_Substitution + VARIANT_Insertion + VARIANT_Deletion
    VARIANT_Others = int(len(df)) - int(sumvar)
    x.append(VARIANT_Others)
    
    #IMPACT
    #"IMPACT_Modifier" - "IMPACT_Low" - "IMPACT_Moderate" - "IMPACT_HIGH"
    IMPACT_Modifier = len(df[df["IMPACT"]=="MODIFIER"])
    x.append(IMPACT_Modifier)
    IMPACT_Low = len(df[df["IMPACT"]=="LOW"])
    x.append(IMPACT_Low)
    IMPACT_Moderate = len(df[df["IMPACT"]=="MODERATE"])
    x.append(IMPACT_Moderate)
    IMPACT_High = len(df[df["IMPACT"]=="HIGH"])
    x.append(IMPACT_High)

    #SIFT
    df["SIFT_score"] = df.SIFT.apply(lambda x: x.split("(")[-1][:-1] if(x!= "-") else x)
    SIFT_DEL = sum(df[df.SIFT_score != "-"].SIFT_score.astype(float) <= 0.05)
    x.append(SIFT_DEL)
    SIFT_NOT_DEL = sum(df[df.SIFT_score != "-"].SIFT_score.astype(float) > 0.05)
    x.append(SIFT_NOT_DEL)

    #POLYPHEN
    df["POLY_score"] = df.PolyPhen.apply(lambda x: x.split("(")[0] if(x!= "-") else x)
    POLYPHEN_unknown = sum(df[df.POLY_score != "-"]["POLY_score"]=="unknown")
    x.append(POLYPHEN_unknown)
    POLYPHEN_benign = sum(df[df.POLY_score != "-"]["POLY_score"]=="benign")
    x.append(POLYPHEN_benign)
    POLYPHEN_possibly_damaging = sum(df[df.POLY_score != "-"]["POLY_score"]=="possibly_damaging")
    x.append(POLYPHEN_possibly_damaging)
    POLYPHEN_probably_damaging = sum(df[df.POLY_score != "-"]["POLY_score"]=="probably_damaging")
    x.append(POLYPHEN_probably_damaging)
    
    #GERP
    GERP_constrained = sum(df[df["GERP++_RS"] != "-"]["GERP++_RS"].astype(float) >=  2.0)
    x.append(GERP_constrained)
    GERP_not_constrained = sum(df[df["GERP++_RS"] != "-"]["GERP++_RS"].astype(float) < 2.0)
    x.append(GERP_not_constrained)

    #CADD
    CADD_DEL = sum(df[df["CADD_phred"] != "-"]["CADD_phred"].astype(float) >= 30.0)
    x.append(CADD_DEL)
    CADD_NOT_DEL = sum(df[df["CADD_phred"] != "-"]["CADD_phred"].astype(float) <= 30.0)
    x.append(CADD_NOT_DEL)
    
    if(i==0):
        final_df = pd.DataFrame(data=[x],columns=[columns_names])
        i=i+1
    else:
        final_df.loc[len(final_df)] = x

=== LOADING INFO_Subsample_007pct_BWA_5X.bam_FreeBayes.vcf_vep.txt ===
=== Treating INFO_Subsample_007pct_BWA_5X.bam_FreeBayes.vcf_vep.txt ===
=== LOADING INFO_Subsample_007pct_VG_5X.bam_FreeBayes.vcf_vep.txt ===
=== Treating INFO_Subsample_007pct_VG_5X.bam_FreeBayes.vcf_vep.txt ===
=== LOADING INFO_Subsample_014pct_BWA_10X.bam_FreeBayes.vcf_vep.txt ===
=== Treating INFO_Subsample_014pct_BWA_10X.bam_FreeBayes.vcf_vep.txt ===
=== LOADING INFO_Subsample_014pct_VG_10X.bam_FreeBayes.vcf_vep.txt ===
=== Treating INFO_Subsample_014pct_VG_10X.bam_FreeBayes.vcf_vep.txt ===
=== LOADING INFO_Subsample_028pct_BWA_20X.bam_FreeBayes.vcf_vep.txt ===
=== Treating INFO_Subsample_028pct_BWA_20X.bam_FreeBayes.vcf_vep.txt ===
=== LOADING INFO_Subsample_028pct_VG_20X.bam_FreeBayes.vcf_vep.txt ===
=== Treating INFO_Subsample_028pct_VG_20X.bam_FreeBayes.vcf_vep.txt ===
=== LOADING INFO_Subsample_042pct_BWA_30X.bam_FreeBayes.vcf_vep.txt ===
=== Treating INFO_Subsample_042pct_BWA_30X.bam_FreeBayes.vcf_vep.txt

In [9]:
final_df.to_csv('Final_DF2.csv',sep=',', header=True)

In [14]:
xxx = pd.read_csv("Final_DF.csv")

In [18]:
xxx

,Sample,Method,Coverage,Age,NB_total_variant,VARIANT_SNV,VARIANT_INDEL,VARIANT_Substitution,VARIANT_Insertion,VARIANT_Deletion,...,SIFT_deletarious,SIFT_not_deletarious,POLYPHEN_unknown,POLYPHEN_benign,POLYPHEN_possibly_damaging,POLYPHEN_probably_damaging,GERP_constrained,GERP_not_constrained,CADD_deletarious,CADD_not_deletarious
0,ATP016,VG-bam,IDK,IDKy,27492877,24300684,2413729,778464,0,0,...,50692,64342,2111,68241,18147,28879,84292,31906,10737,106823
1,ATP016,VG-gam,IDK,IDKy,22763210,19842721,33778,284129,973497,1590741,...,12906,37139,1409,39512,5666,4767,23032,23720,552,47357
2,ATP016,BWA1-bam,IDK,IDKy,28509934,25828048,1767775,914111,0,0,...,60165,70790,2279,75871,20550,34903,99563,33665,12808,120928
3,BOT2016,VG-bam,IDK,IDKy,17945680,15038337,2525810,381533,0,0,...,15301,32430,977,34013,5671,8313,29458,17385,2565,44680
4,BOT2016,VG-gam,IDK,IDKy,15238023,12965862,24919,151086,912931,1154701,...,6775,26128,850,27073,3354,2554,14041,16101,237,30994
5,BOT2016,BWA1-bam,IDK,IDKy,17338750,15210178,1763306,365266,0,0,...,16073,31997,968,33635,5742,8977,31046,16705,2769,45180
6,BR2,VG-bam,IDK,IDKy,16489220,13991960,2110295,386965,0,0,...,13172,29724,981,31548,5164,6176,22615,17661,1381,39715
7,BR2,VG-gam,IDK,IDKy,18639327,16203843,29520,202086,936146,1235398,...,6670,26406,969,27428,3217,2335,13655,16573,274,30979
8,BR2,BWA1-bam,IDK,IDKy,15711935,13893352,1362135,456448,0,0,...,13172,28422,936,30575,4778,6276,23210,16638,1486,38560
9,JP14,VG-bam,IDK,IDKy,16918052,13858002,2590982,469068,0,0,...,16398,32978,1081,34747,6097,8517,27525,19305,2226,45436


In [222]:
final_df

,Sample,Method,Coverage,NB_total_variant,VARIANT_SNV,VARIANT_INDEL,VARIANT_Substitution,VARIANT_Insertion,VARIANT_Deletion,VARIANT_Others,...,SIFT_deletarious,SIFT_not_deletarious,POLYPHEN_unknown,POLYPHEN_benign,POLYPHEN_possibly_damaging,POLYPHEN_probably_damaging,GERP_constrained,GERP_not_constrained,CADD_deletarious,CADD_not_deletarious
0,ATP016,VG-bam,IDK,27492877,24300684,2413729,778464,0,0,0,...,50692,64342,2111,68241,18147,28879,84292,31906,10737,106823
1,ATP016,VG-gam,IDK,22763210,19842721,33778,284129,973497,1590741,38344,...,12906,37139,1409,39512,5666,4767,23032,23720,552,47357


# Find name

In [181]:
Name = file.split("INFO_")[-1].split("_")[0]

# Find method

In [194]:
method = file.split("INFO_")[-1].split("_")[2]

'VG-gam'

# Sample name

In [95]:
x=[]
x.append(Name)

# Method

In [96]:
x.append(method)

# Coverage

In [97]:
x.append("IDK")

# NB_total_variant

In [98]:
x.append(int(len(df)))

# Variant Type

"VARIANT_SNV" - "VARIANT_INDEL" - "VARIANT_Substitution" - "VARIANT_Insertion" - "VARIANT_Deletion" - "VARIANT_Others"

In [99]:
VARIANT_SNV = len(df[df["VARIANT_CLASS"]=="SNV"])
x.append(VARIANT_SNV)

In [100]:
VARIANT_INDEL = len(df[df["VARIANT_CLASS"]=="indel"])
x.append(VARIANT_INDEL)

In [101]:
VARIANT_Substitution = len(df[df["VARIANT_CLASS"]=="substitution"])
x.append(VARIANT_Substitution)

In [102]:
VARIANT_Insertion = len(df[df["VARIANT_CLASS"]=="insertion"])
x.append(VARIANT_Insertion)

In [103]:
VARIANT_Deletion = len(df[df["VARIANT_CLASS"]=="deletion"])
x.append(VARIANT_Deletion)

In [104]:
sumvar = VARIANT_SNV + VARIANT_INDEL + VARIANT_Substitution + VARIANT_Insertion + VARIANT_Deletion
VARIANT_Others = int(len(df)) - int(sumvar)
x.append(VARIANT_Others)

# IMPACT

"IMPACT_Modifier" - "IMPACT_Low" - "IMPACT_Moderate" - "IMPACT_HIGH"

In [111]:
IMPACT_Modifier = len(df[df["IMPACT"]=="MODIFIER"])
x.append(IMPACT_Modifier)

In [112]:
IMPACT_Low = len(df[df["IMPACT"]=="LOW"])
x.append(IMPACT_Low)

In [113]:
IMPACT_Moderate = len(df[df["IMPACT"]=="MODERATE"])
x.append(IMPACT_Moderate)

In [114]:
IMPACT_High = len(df[df["IMPACT"]=="HIGH"])
x.append(IMPACT_High)

# SIFT

"SIFT_deletarious" - "SIFT_not_deletarious"

predicts whether an amino acid substitution is likely to affect protein function (based on sequence homology and the physico-chemical similarity between the alternate amino acids) The qualitative prediction is derived from this score such that substitutions with a score < 0.05 are called 'deleterious' and all others are called 'tolerated'. i.e. (tolerated(0.38),deleterious(0.03))

In [116]:
df["SIFT_score"] = df.SIFT.apply(lambda x: x.split("(")[-1][:-1] if(x!= "-") else x)

In [120]:
SIFT_DEL = sum(df[df.SIFT_score != "-"].SIFT_score.astype(float) <= 0.05)
x.append(SIFT_DEL)

In [121]:
SIFT_NOT_DEL = sum(df[df.SIFT_score != "-"].SIFT_score.astype(float) > 0.05)
x.append(SIFT_NOT_DEL)

# POLYPHEN

"POLYPHEN_unknown" - "POLYPHEN_benign" - "POLYPHEN_possibly_damaging" - "POLYPHEN_probably_damaging"

In [130]:
df["POLY_score"] = df.PolyPhen.apply(lambda x: x.split("(")[0] if(x!= "-") else x)

In [131]:
POLYPHEN_unknown = sum(df[df.POLY_score != "-"]["POLY_score"]=="unknown")
x.append(POLYPHEN_unknown)

In [132]:
POLYPHEN_benign = sum(df[df.POLY_score != "-"]["POLY_score"]=="benign")
x.append(POLYPHEN_benign)

In [133]:
POLYPHEN_possibly_damaging = sum(df[df.POLY_score != "-"]["POLY_score"]=="possibly_damaging")
x.append(POLYPHEN_possibly_damaging)

In [134]:
POLYPHEN_probably_damaging = sum(df[df.POLY_score != "-"]["POLY_score"]=="probably_damaging")
x.append(POLYPHEN_probably_damaging)

# GERP

"GERP_constrained" - "GERP_not_constrained"

In [149]:
GERP_constrained = sum(df[df["GERP++_RS"] != "-"]["GERP++_RS"].astype(float) >=  2.0)
x.append(GERP_constrained)

In [150]:
GERP_not_constrained = sum(df[df["GERP++_RS"] != "-"]["GERP++_RS"].astype(float) < 2.0)
x.append(GERP_not_constrained)

# CADD

"CADD_deletarious" - "CADD_not_deletarious"

Variants with scores over 30 are predicted to be the 0.1% most deleterious possible substitutions in the human genome.

In [159]:
CADD_DEL = sum(df[df["CADD_phred"] != "-"]["CADD_phred"].astype(float) >= 30.0)
x.append(CADD_DEL)

In [160]:
CADD_NOT_DEL = sum(df[df["CADD_phred"] != "-"]["CADD_phred"].astype(float) <= 30.0)
x.append(CADD_NOT_DEL)

In [165]:
len(x)

24

In [166]:
len(columns_names)

24

In [168]:
xx = pd.DataFrame(data=[x],columns=[columns_names])

In [170]:
rz = x

In [175]:
xx.loc[len(xx)] = rz